In [82]:
import anthropic

In [83]:
from dotenv import load_dotenv

In [84]:
load_dotenv()

True

In [85]:
client_anthropic = anthropic.Anthropic(api_key = os.getenv('CLAUDE_API_KEY'))

In [86]:
import os

In [87]:
import json

In [88]:
from supabase import create_client, Client

In [89]:
supabase_api_key = os.environ.get("SUPBASE_KEY")
supabase_url = os.environ.get("SUPABASE_URL")

In [90]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [91]:
researched_data = [{'event_id': '299_2026-02-08',
  'news_date': '2026-02-08',
  'output': '- Verified: The earbuds rumor centers on an internal code name for OpenAI’s first consumer hardware device being Sweetpea. Multiple credible outlets report this codename as part of the leak cycle about the device. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: Several credible outlets note that the project is not officially confirmed and remains a leak/rumor at this time. TechCrunch explicitly describes the details as coming from leaks and “tipsters,” not an official OpenAI confirmation; Axios likewise characterizes the information as rumors shared at Davos. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: The same reporting that mentions Sweetpea also references a potential second consumer device codenamed Gumdrop in the rumor mill. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Added confirmed hardware context: OpenAI disclosed at Davos that it is building a new AI device, though details remain scarce. Timeline given by OpenAI officials in these discussions points to a likely release in the second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Confirmed: OpenAI’s prior hardware efforts are linked to its 2025 acquisition of Jony Ive’s design firm io (the deal was reported to be around $6.5 billion). Ive is associated with leading the hardware design work for the product family. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: OpenAI’s hardware plans discussed publicly have included a screen-free, pocketable device concept as part of the early product narrative. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: Reported design descriptions for the Sweetpea concept include a main unit described as metal and egg-shaped, with two pill-shaped modules that would sit behind the ear. These details appear in multiple outlets drawing on the same leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (design details corroborated by multiple outlets): The rumored hardware would reportedly use a 2-nanometer-class processor (with Exynos as a favored reference) for on-device AI tasks, aligned with the “phone-like” compute ambitions discussed in the leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (manufacturing partner context): Reports mention Foxconn as a manufacturing partner for the OpenAI devices, with some mention of Luxshare as a possibility; this is described in the leakage coverage and echoed by outlets citing Asian reporting. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: First-year shipment targets cited in the leaks are in the 40–50 million units range, with September 2026 as a possible launch window in at least some outlets. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed (scope/roadmap detail in leaks): Some leakage coverage claims OpenAI is planning a broader hardware roadmap (up to five devices by 2028), with Sweetpea at the front of that lineup. This detail appears in multiple leak-focused outlets. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n\n- Not corroborated by credible outlets (branding name caveat): The claim that the consumer-facing name will be “Dime” lacks confirmation from established major outlets. Some blogs and tech sites have floated “Dime” as branding based on patent filings or rumor chatter, but there is no widely credited report in outlets like TechCrunch, Axios, or The Indian Express confirming “Dime.” Examples of circulating but less-established sources include Gizmochina and other aggregators. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\n- Note on the current status of confirmation: The available reporting from January–February 2026 consistently describes the information as leaks/rumors rather than official confirmation; OpenAI executives acknowledged work on a hardware device but kept specifics (design, branding, features) under wraps. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nSources referenced above (examples of main reporting):\n- TechCrunch: OpenAI aims to ship its first device in 2026, and it could be earbuds (codename Sweetpea; design, chip, and unit targets discussed). ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n- Indian Express: OpenAI’s first product could be ChatGPT-powered earbuds; codename Sweetpea; design details and 2nm Samsung Exynos chip mentioned. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n- Android Authority: OpenAI earbuds leak with Sweetpea codename; design description; BOM/production cost hints; September launch target; five-device roadmap mentioned. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n- Axios: OpenAI’s secret device coming; Davos timeline; mentions Sweetpea and Gumdrop leaks; timeline for second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n- The Verge: OpenAI hardware prototype discussions tied to Jony Ive and the hardware program (context around the hardware push). ([theverge.com](https://www.theverge.com/news/827607/openai-hardware-prototype-chatgpt-jony-ive-sam-altman?utm_source=openai))\n- (Caveat on Dime branding) Gizmochina / other aggregators noting a claimed consumer name Dime; these are not widely corroborated by major outlets. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\nNotes:\n- All statements above about Sweetpea, the rumored features, and the general timeline are based on published reporting as of February 8, 2026; several items (notably the consumer-name “Dime”) are not yet verified by major, high-profile outlets and are reported only by a subset of non-primary sources. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nIf you’d like, I can add direct excerpts or align these points to a formal, source-by-source bibliography.'}]

In [93]:
result = supabase.table('research_assistant')\
    .select('id, event_id, news_date, output')\
.eq('news_date', '2026-02-20')\
    .order('created_at', desc=False)\
    .execute()

In [94]:
result.data

[{'id': 211,
  'event_id': '1274_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Core development: Nvidia is close to finalizing roughly a $30 billion equity investment in OpenAI, replacing the previously announced up-to-$100 billion multi-year data-center compute pact. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Financing scope: OpenAI is seeking more than $100 billion in the current funding round, with the round valuing OpenAI at about $830 billion. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Likely investors: SoftBank and Amazon are also expected to participate in the round. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft

In [95]:
final_summary_7 = []

In [102]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a tech journalist at Krux — a news platform that helps GenZ stay informed about AI & tech in under 60 seconds.

                CRITICAL CONSTRAINT: The article body MUST be 85-100 words (excluding headline). Count carefully before finalizing.

                TASK: Write a news article using ONLY the information provided by our research assistant.
                
                Follow this structure:
                
                === STEP 1: HEADLINE (Under 10 words) ===
                Objective: Hook the reader instantly
                - Punchy, attention-grabbing and clickbait
                - Focus on the most surprising/important angle
                
                **ARTICLE BODY** (85-100 words TOTAL)

                Opening (20-30 words):
                - If rumour: clarify in first sentence, then proceed assuming the things stated by research assistant are facts
                - Lead with the core news hook
                
                Middle (40-50 words):
                - Sharp, conversational, slightly witty tone
                - Present key facts from research assistant
                - Use most/all provided information
                - NO links or source references in body
                - Explain it in a simple way such that a non-tech person who has no background about this also understands its impact
                
                Closing (20-25 words):
                - Forward-looking strategic perspective
                - What could this mean for industry/users/tech landscape?
                - Frame as analysis, NOT opinion
                - Grounded in facts presented
                
                === VERIFICATION CHECKLIST ===
                Before outputting, verify:
                ☑ Article body is 85-100 words (count manually)
                ☑ Headline is under 10 words
                ☑ No URLs in article body
                ☑ All key facts from research included
                ☑ There should be no em-dashes in the article
                
                HARD CONSTRAINTS:
                - Total article: 100 words max (headline excluded). If the Article is NOT 100 words, re-generate it to meet the 100 word
                requirement.
                - Sources with URLs listed at end under "Sources:"

                
                OUTPUT FORMAT NEEDS TO BE IN A STRICT JSON FORMAT:
                {{"headline": "...", "output": "...", "sources": [{{"name": "...", "url": "..."}}]}}

                Here is the information we have gathered from the research assistant:
                
                {i['output']}
            """
            }]
        )

        final_output = response.content[0].text

        cleaned = final_output.strip()

        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        article_json = json.loads(cleaned)

        article_json['model_provider'] = 'claude'
        article_json['news_date'] = i['news_date']
        article_json['event_id'] = i['event_id']

        print(article_json)

        final_summary_7.append(article_json)

        # save_article(article_json)

        print("✔️ Saved!")
        
        

    #     final_news_article.append(json_format)

    # return final_news_article

In [103]:
a = [{
  "id": 223,
  "event_id": "1291_2026-02-20",
  "news_date": "2026-02-20",
  "output": "- Core development: Lyria 3, Google DeepMind's latest generative music model, is rolling out in beta inside the Gemini app, enabling creation of 30-second tracks from text prompts, photos, or videos, with auto lyrics and optional cover art; output is watermark-tagged via SynthID. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Global beta availability and rollout: The feature is available to Gemini users 18+, with desktop access today and mobile rollout in the coming days; supported languages include English, German, Spanish, French, Hindi, Japanese, Korean, and Portuguese. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Output length and scope: Lyria 3 targets 30-second tracks, designed for short-form content (e.g., YouTube Shorts, TikTok, Reels). ([techcrunch.com](https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/?utm_source=openai))\n\n- Auto lyrics capability: Lyrics generation is built into the flow, removing the previous requirement to supply lyrics manually. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Multimodal control: Users can steer generation with text plus visual prompts (images/videos) to influence genre, mood, tempo, and vocal tone. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Copyright and safety safeguards: Outputs are filtered to discourage direct artist mimicry, with provenance tools and a reporting channel for rights concerns; SynthID watermarking supports identification of AI-generated audio. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- YouTube Dream Track integration: Lyria 3 is being extended to YouTube creators via Dream Track, expanding access beyond Gemini to creator workflows. ([techcrunch.com](https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/?utm_source=openai))\n\n- Creator-facing features (art and branding): Gemini generates custom cover art for tracks (e.g., via Nano Banana) to accompany AI-made songs. ([techcrunch.com](https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/?utm_source=openai))\n\n- Availability and access tier details: Free usage exists with plan-based limits; Google AI Plus, Pro, and Ultra subscribers are described as receiving higher generation ceilings. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai)) - Google DeepMind launched Lyria 3, its most capable generative music model to date, in beta within the Gemini app on February 18, 2026, enabling users to generate music tracks complete with vocals and lyrics from text, photo, or video prompts.\n\n- Lyria 3 produces dynamic 30-second tracks with crystal-clear 48kHz stereo audio.\n\n- Beta rollout began on Gemini desktop that day, expanding to the Gemini app globally over the following days for all users aged 18+ across 8 languages.\n\n- Users access the feature via \"Create Music\" in the Gemini tools menu or directly at gemini.google.com/music.\n\n- Provides granular controls including tempo settings, specific vocal styles, and precise lyrics input.\n\n- Every generated track embeds SynthID, Google's invisible watermark for AI content identification.\n\n- Lyria 3 is also available via YouTube's Dream Track for creators in the US, with expansion to more countries planned.\n\n- Model designed for original expression; prompts naming artists provide broad stylistic inspiration rather than direct mimicry.",
  "model_provider": 'openai'
}]

In [104]:
claude_article_builder(a)

{'headline': "Google's Lyria 3 Turns Text Into 30-Second Songs", 'output': "Google DeepMind just launched Lyria 3, a music generation AI inside the Gemini app that creates 30-second tracks with vocals and lyrics from text, photos, or videos. Rolling out globally to users 18+ in eight languages, it targets short-form content like YouTube Shorts and TikTok. The tool auto-generates lyrics, lets you tweak tempo and vocal style, and even makes custom cover art. Every track gets SynthID watermarking to flag AI-made audio. Available free with usage caps, it's also powering YouTube's Dream Track for creators. This could reshape how anyone makes shareable audio, lowering the barrier from studio to smartphone.", 'sources': [{'name': 'Google Blog', 'url': 'https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai'}, {'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/?utm_source=openai'}], 'model_pr

In [100]:
def save_article(article_json):
    supabase.table('hundred_word_articles').insert({
        'event_id': article_json['event_id'],
        'model_provider': article_json['model_provider'],
        'news_date': article_json['news_date'],
        'sources': json.dumps(article_json['sources']),
        'output': article_json['output'],
        'headline': article_json['headline']
    }).execute()

In [115]:
final_summary_7

[{'headline': 'Nvidia Ditches $100B Chip Deal for $30B OpenAI Stake',
  'output': "Nvidia is swapping its massive $100 billion multi-year chip supply agreement for a cleaner $30 billion direct equity investment in OpenAI. This shift makes Nvidia an actual owner, not just a supplier, in the AI giant currently valued at $830 billion. OpenAI's hunting for over $100 billion total this round, with SoftBank and Amazon joining the party. Plot twist: OpenAI plans to spend much of that fresh cash buying Nvidia chips anyway. This move signals Nvidia wants skin in the game beyond hardware sales, positioning itself as a stakeholder in AI's future rather than just the picks-and-shovels provider during the gold rush.",
  'sources': [{'name': 'The Star',
    'url': 'https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports'}],
  'model_provider': 'claude',
  'news_date': '2026-02-20',
  'event_id': '1274_2026-02-20

In [81]:
for i in final_summary_7:
    if i['news_date'] >= '2026-02-17':
        save_article(i)
        print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [25]:
for i in final_summary_3:
    save_article(i)
    print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [110]:

fail = []
pas = 0

for i in final_summary_7:
    words = i['output'].split()
    word_count = len(words)
    if word_count > 100:
        print(f"Fail: {word_count}")
        fail.append(word_count)
    else:
        print(f"Pass: {word_count}")
        pas +=1

print(f"Fail: {len(fail)/len(final_summary_7)}")



Fail: 102
Fail: 107
Pass: 100
Pass: 100
Pass: 97
Fail: 101
Fail: 104
Pass: 97
Fail: 104
Pass: 97
Pass: 95
Pass: 100
Fail: 109
Pass: 100
Fail: 104
Pass: 99
Fail: 0.4375


In [223]:
import statistics as stats

In [111]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a tech journalist at Krux — a news platform that helps GenZ stay informed about AI & tech in under 60 seconds.

                CRITICAL CONSTRAINT: The article body MUST be 85-100 words (excluding headline). Count carefully before finalizing.

                TASK: Write a news article using ONLY the information provided by our research assistant. The research assistant will give 
                you outputs from 2 different types:
                1. Blogs/Articles/etc
                2. X/Twitter

                You need to ingest everything and take out the most relevant points to carefully write an article. By following this structure:
                
                === STEP 1: HEADLINE (Under 10 words) ===
                Objective: Hook the reader instantly
                - Punchy, attention-grabbing and clickbait
                - Focus on the most surprising/important angle
                
                **ARTICLE BODY** (85-100 words TOTAL)

                Opening (20-30 words):
                - If rumour: clarify in first sentence, then proceed assuming the things stated by research assistant are facts
                - Lead with the core news hook
                
                Middle (40-50 words):
                - Sharp, conversational, slightly witty tone
                - Present key facts from research assistant
                - Use most/all provided information
                - NO links or source references in body
                - Explain it in a simple way such that a non-tech person who has no background about this also understands its impact
                
                Closing (20-25 words):
                - Forward-looking strategic perspective
                - What could this mean for industry/users/tech landscape?
                - Frame as analysis, NOT opinion
                - Grounded in facts presented
                
                === VERIFICATION CHECKLIST ===
                Before outputting, verify:
                ☑ Article body is 85-100 words (count manually)
                ☑ Headline is under 10 words
                ☑ No URLs in article body
                ☑ All key facts from research included
                ☑ There should be no em-dashes in the article
                
                HARD CONSTRAINTS:
                - Total article: 100 words max (headline excluded). If the Article is NOT 100 words, re-generate it to meet the 100 word
                requirement.
                - Sources with URLs listed at end under "Sources:"

                
                OUTPUT FORMAT NEEDS TO BE IN A STRICT JSON FORMAT:
                {{"headline": "...", "output": "...", "sources": [{{"name": "...", "url": "..."}}]}}

                Here is the information we have gathered from the research assistant:

                
                Research output from news/blogs: {i['deep_research_output']}

                Research output from X/Twitter: {i['x_output']}
            """
            }]
        )

        final_output = response.content[0].text

        cleaned = final_output.strip()

        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        article_json = json.loads(cleaned)

        article_json['model_provider'] = 'claude'
        article_json['news_date'] = i['news_date']
        article_json['event_id'] = i['event_id']

        print(article_json)

        final_summary_7.append(article_json)

        # save_article(article_json)

        print("✔️ Saved!")
        
        

    #     final_news_article.append(json_format)

    # return final_news_article

In [118]:
test = [{
    "id": 224,
    "event_id": "1291_2026-02-20",
    "news_date": "2026-02-20",
    "x_output": "- Sarvam AI released two new large language models, Sarvam 30B and Sarvam 105B, on February 18, 2026.\n\n- Both models use Mixture of Experts (MoE) architecture and are built from scratch.\n\n- Sarvam 30B activates 1B non-embedded parameters per token.\n\n- Sarvam 30B was pretrained on 16 trillion tokens spanning code, web, multilingual, and mathematical data.\n\n- Sarvam 30B supports a 32K context window for long-running agentic interactions.\n\n- Sarvam 30B targets real-time applications like conversational AI and high-throughput workflows.\n\n- Sarvam 105B activates 9B parameters per token.\n\n- Sarvam 105B supports a 128K context window.\n\n- Sarvam 105B handles complex reasoning, agentic task completion, tool use, coding, mathematics, and science.\n\n- Sarvam AI builds full-stack generative AI for population-scale use.",
    "deep_research_output": "- Core development: Sarvam AI unveiled two indigenous LLMs, Sarvam-30B and Sarvam-105B, announced at the India AI Impact Summit 2026 in New Delhi, aimed at real-time use and advanced reasoning; MoE architecture with large context windows; open-sourcing planned. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Model architecture: Both models use a mixture-of-experts design that activates only a fraction of parameters at inference to improve efficiency. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Real-time capabilities: 30B targets production-ready, real-time conversational use with a 32,000-token context window; 105B targets more complex, multi-step reasoning with a 128,000-token window. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Training from scratch: 30B was pre-trained from scratch on about 16 trillion tokens; 105B trained on trillions of tokens across multiple Indian languages. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- IndiaAI Mission and compute: Training leveraged India's government-backed IndiaAI Mission compute, with infrastructure support from Yotta and technical support from Nvidia. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Open-source plan: Sarvam stated it intends to open-source the 30B and 105B models, though it did not specify whether training data or full training code would be disclosed. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Enterprise and ecosystem: The launch outlined an expanded enterprise stack, including Sarvam for Work and a conversational agent platform called Samvaad, plus a vision model for document understanding. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Indus consumer app: Two days after the model announcements, Sarvam released Indus, a consumer AI chat app powered by the 105B model, available in beta on iOS, Android, and the web; initial rollout limited to India with a waitlist. ([techcrunch.com](https://techcrunch.com/2026/02/20/indias-sarvam-launches-indus-ai-chat-app-as-competition-heats-up/?utm_source=openai))\n\n- Benchmark and language focus: The 105B model is positioned to perform on benchmarks relevant to Indian languages and enterprise tasks, with claims of competitive performance against open/closed models of similar size. ([business-standard.com](https://www.business-standard.com/technology/tech-news/sarvam-105b-model-sovereign-ai-india-foundation-model-launch-impact-summit-126021900551_1.html?utm_source=openai))\n\n- Industry coverage: Coverage of the announcement and related products appeared in TechCrunch (Feb 18, 2026), Economic Times (ETtech, Feb 18, 2026), Fortune India (Feb 18, 2026), and The Times of India (Feb 19, 2026), among others. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))"
  }]

In [119]:
claude_article_builder(test)

{'headline': 'Indian Startup Sarvam AI Drops Two Homegrown LLMs', 'output': "Sarvam AI just unveiled two indigenous large language models, Sarvam-30B and Sarvam-105B, at India's AI Impact Summit 2026. Both were trained from scratch on trillions of tokens across multiple Indian languages using government-backed compute infrastructure. The 30B model targets real-time conversations with a 32,000-token context window, while the 105B handles complex reasoning with a massive 128,000-token window. Both use mixture-of-experts architecture, activating only needed parameters for efficiency. Sarvam plans to open-source both models and launched Indus, a consumer chat app powered by the 105B. This positions India to compete in the global AI race with sovereign, multilingual models tailored for local needs.", 'sources': [{'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/'}, {'name': 'TechCrunch', 'url'

In [ ]:
[
  {
    "headline": "India's Sarvam AI Drops Two Homegrown LLMs, Goes Open-Source",
    "output": "Sarvam AI just unveiled two made-in-India language models at the India AI Impact Summit: 
    Sarvam-30B for real-time chat and Sarvam-105B for complex reasoning. Both use mixture-of-experts architecture, 
    activating only some parameters to stay efficient while handling massive context windows (32K and 128K tokens). 
    Trained from scratch on trillions of tokens across Indian languages using government-backed compute, they 
    rival global models but understand local languages better. Sarvam plans to open-source both and launched Indus, 
    a consumer chat app powered by 105B. This could make advanced AI accessible across India's linguistic diversity, 
    challenging Western-dominated models with sovereign, multilingual alternatives."
  },
  {
    "headline": "Stanford and AWS Team Up for AI Marketing Lab",
    "output": "Stanford HAI and AWS just launched the Stanford and AWS Marketing Science Lab to revolutionize how companies measure marketing effectiveness using AI. The collaboration brings together Stanford's Institute for Human-Centered AI, Data Science department, Graduate School of Business, and AWS cloud infrastructure. Led by top economists including Nobel laureate Guido Imbens and Susan Athey, the lab will tackle causal inference, AI measurement tools, and B2B customer journey analysis. Think smarter ad spend decisions powered by algorithms instead of guesswork. The initiative plans to release published research, open-source code, and prototypes, potentially reshaping how businesses justify their marketing budgets and adopt AI tools at scale."
  },
  {
    "headline": "ChatGPT's New Lockdown Mode Blocks Live Web Browsing",
    "output": "OpenAI just dropped Lockdown Mode for ChatGPT, a high-security setting rolling out to Enterprise, Edu, Healthcare, and Teachers accounts, with consumer access coming soon. Designed for executives and security teams worried about data leaks, it tackles prompt-injection attacks by blocking live web requests and restricting external app access. Instead of browsing the open internet, ChatGPT can only pull cached content within OpenAI's network. Admins can customize which tools stay active per user. Paired with new Elevated Risk labels flagging dangerous features, this upgrade could reshape how organizations trust AI with sensitive workflows and confidential information."
  },
  {
    "headline": "Google's Lyria 3 Turns Text Into Music in Gemini",
    "output": "Google DeepMind just dropped Lyria 3 in beta, letting Gemini users create 30-second tracks from text, photos, or videos with auto-generated lyrics and cover art. Think of it as Instagram for your ears: describe a vibe or upload a sunset pic, and get a custom soundtrack perfect for TikTok or Reels. The AI handles genre, mood, tempo, and vocals while blocking artist copycats. Each track gets watermarked via SynthID so you can spot AI audio. Available now on desktop (mobile coming soon) for users 18+ in eight languages, with YouTube Dream Track integration expanding creator access. As AI music tools proliferate, expect short-form content to become hyper-personalized sonic experiences."
  },
  {
    "headline": "Insurance Startup Qumis Raises $4.3M for Lawyer-Level AI",
    "output": "Chicago-based Qumis just closed an oversubscribed $4.3 million seed round led by MTech Capital, bringing total funding to $6.75 million. The startup uses attorney-trained AI to read complex commercial insurance policies and deliver citation-backed insights with transparent reasoning, replacing tedious manual reviews. Major brokers like NFP, an Aon company, have already scaled from pilot teams to hundreds of users. While most insurance AI tackles workflow automation, Qumis is betting on specialized coverage intelligence. The funds will expand its go-to-market team and deepen product capabilities, positioning it as AI moves beyond document processing into nuanced legal interpretation for underwriting and claims."
  },
  {
    "headline": "OpenAI and Tata Group Join Forces for India's AI Revolution",
    "output": "OpenAI and Tata Group have announced a massive strategic partnership to accelerate AI transformation across India and beyond. The deal includes deploying ChatGPT Enterprise to hundreds of thousands of TCS employees, building sovereign AI data centers starting at 100 MW (scalable to 1 GW), and developing industry-specific agentic AI solutions. TCS becomes OpenAI's first HyperVault customer, ensuring data stays local and secure. The partnership also brings 100,000 ChatGPT Edu licenses and AI training for one million Indian youth. With new OpenAI offices opening in Mumbai and Bengaluru by 2026, this positions India as a global AI powerhouse while giving enterprises tools to compete in the AI-first economy."
  },
  {
    "headline": "Fractal's Vaidya 2.0 Beats GPT-5 on Healthcare Benchmark",
    "output": "Fractal just launched Vaidya 2.0, a healthcare AI model that's crushing the competition. It scored 50.1 on OpenAI's notoriously tough HealthBench test, becoming the first model to cross 50 and outperforming both GPT-5 and Google's Gemini Pro 3. Think of it as an AI doctor's assistant that can triage emergencies, check symptoms, and guide patients through treatment. Unlike typical AI models that just recall medical facts, Vaidya 2.0 actually reasons through complex healthcare scenarios. Part of India's $1.2 billion AI push, it's designed specifically for healthcare systems in developing regions, bridging the gap between raw patient data and actual medical action."
  },
  {
    "headline": "OpenAI Building Smart Speakers, Glasses After $6.5B Acquisition",
    "output": "OpenAI is jumping into consumer hardware with a team of over 200 people developing AI-powered devices, including smart speakers, glasses, and even a smart lamp. This follows their massive $6.5 billion acquisition of io Products, Jony Ive's design firm. The first product, a camera-equipped smart speaker priced at $200 to $300, won't ship until February 2027 at the earliest, while smart glasses are slated for 2028. This pivot from pure software to physical products puts OpenAI in direct competition with Meta's successful Ray-Ban glasses and upcoming wearables from Apple and Google, as AI companies race to embed intelligence into everyday objects around your home."
  },
  {
    "headline": "OpenAI and Microsoft Join UK's £27M AI Safety Push",
    "output": "OpenAI and Microsoft have officially joined the UK's AI Security Institute's international coalition to keep AI development safe and under human control. The Alignment Project, a UK-led initiative, now boasts over £27 million in funding supporting roughly 60 research projects across eight countries. OpenAI is chipping in £5.6 million, joining partners like Anthropic, AWS, and Canada's CIFAR. Think of it as a global insurance policy: researchers get grants, computing power, and mentorship to ensure future AI systems don't go rogue. With a second funding round opening this summer, the coalition aims to build public trust and enable safer AI deployment in critical services worldwide."
  },
  {
    "headline": "General Catalyst Drops $5 Billion Bet on India",
    "output": "General Catalyst just pledged $5 billion to India over five years, a massive fivefold jump from its previous $500 million to $1 billion commitment. Announced at the India AI Impact Summit, the funds will flow into AI, healthcare, defense tech, fintech, and consumer startups. GC isn't chasing cutting-edge AI model development. Instead, it's betting on deploying AI solutions at scale in the real world, where India's vast market shines. With CEO Neeraj Arora steering the charge and a portfolio already featuring Zepto and PB Health, this signals India's emergence as a global AI deployment powerhouse, not just a talent hub."
  },
  {
    "headline": "Google Drops Gemini 3.1 Pro With Wild Reasoning Upgrade",
    "output": "Google just launched Gemini 3.1 Pro, a significantly smarter AI model built for complex problems that need more than quick answers. It scored 77.1% on the ARC-AGI-2 benchmark, literally doubling the reasoning power of its predecessor. The upgrade rolls out today across the Gemini app, NotebookLM, API, and enterprise tools. What makes it wild? It can generate animated website code from text, build live aerospace dashboards tracking the ISS, and even code 3D bird simulations with hand-tracking. This positions Google to compete harder in agentic AI workflows where models actually solve multi-step problems autonomously, not just chat."
  },
  {
    "headline": "PhonePe Lets You Pay Friends Just By Talking",
    "output": "PhonePe just launched AI-powered voice and text search built on Microsoft Foundry, letting its 650 million users skip menus entirely. Say \"Pay Hemanth 20 rupees\" and it auto-selects the contact, or ask \"Gold price\" to jump straight there. The feature uses intent-based routing instead of traditional navigation, understanding what you want and taking you there instantly. It runs on a hybrid on-device and cloud model, keeping your transaction data locked within PhonePe's environment. Rolling out in phases across India via the search bar, help center, and history tab, it signals a shift toward conversational interfaces in fintech."
  },
  {
    "headline": "India Gets 8-Exaflop AI Supercomputer From UAE Partnership",
    "output": "Abu Dhabi's G42 and MBZUAI, teaming up with Cerebras and India's C-DAC, just announced an 8-exaflop AI supercomputer for India at the India AI Impact Summit 2026. That's mind-blowing computing power for training massive AI models tailored to Indian needs. The system will be hosted entirely in India with full data sovereignty, meaning all your data stays within national borders under Indian governance. Once live, universities, startups, SMEs, and government ministries get access to this beast. This positions India as a serious player in sovereign AI infrastructure, reducing dependence on foreign cloud giants while accelerating homegrown AI innovation at scale."
  },
  {
    "headline": "Meta's Smartwatch Is Back: 2026 AI Wearable Incoming",
    "output": "Meta is reviving its smartwatch ambitions with Malibu 2, slated for late 2026. The device will pack health tracking and a built-in Meta AI assistant, taking on the Apple Watch directly. This marks a comeback after Meta killed a similar project in 2022 during Reality Labs budget cuts. The smartwatch fits into Meta's broader wearables strategy, potentially syncing with its Ray-Ban smart glasses, which shipped nearly 6 million units in 2025. Meta hasn't officially commented. If successful, the AI-powered watch could anchor an ecosystem where your glasses, wrist, and phone work together seamlessly, pushing wearables beyond fitness into ambient computing."
  },
  {
    "headline": "Nvidia Ditches $100B Chip Deal for $30B OpenAI Stake",
    "output": "Nvidia is swapping its massive $100 billion multi-year chip supply agreement for a cleaner $30 billion direct equity investment in OpenAI. This shift makes Nvidia an actual owner, not just a supplier, in the AI giant currently valued at $830 billion. OpenAI's hunting for over $100 billion total this round, with SoftBank and Amazon joining the party. Plot twist: OpenAI plans to spend much of that fresh cash buying Nvidia chips anyway. This move signals Nvidia wants skin in the game beyond hardware sales, positioning itself as a stakeholder in AI's future rather than just the picks-and-shovels provider during the gold rush."
  }
]